## Week 2

*Welcome back! Scroll down to week 4.*

We first download the required software: LangChain and its dependency `pypdf`

In [ ]:
!pip install --upgrade pip
!pip install --upgrade langchain pypdf

We then load LangChain's `pypdf` loader.

In [ ]:
from langchain.document_loaders import PyPDFLoader

Let's first load our PDF... 

In [ ]:
loader = PyPDFLoader("Data/2021-census-population-occupied-private-dwellings-community-2001-2021.pdf")

In [ ]:
pdf = loader.load_and_split()

Having loaded the PDF, let's now load the CSV. 

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
loader = CSVLoader("Data/Urban_Design_and_Architecture_Awards_Recipients.csv")

In [ ]:
csv = loader.load()

Having loaded our data, we'll now download and load the embedding model.

In [ ]:
!pip install sentence_transformers > /dev/null

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Let's try embedding some text. Observe the output. Once you've tried it, scroll down to continue.

In [ ]:
text = "This is a test document."

In [ ]:
embeddings.embed_query(text)

We now have a working embedding function. Let's install Chroma.

In [ ]:
!pip install -U chromadb

In [ ]:
from langchain.vectorstores import Chroma

Let's make a vector store for our loaded documents!

In [ ]:
db = Chroma.from_documents(pdf, embeddings)

In [ ]:
db.add_documents(csv)

Let's try retrieving a relevant document.

In [ ]:
query = "An award concerning art."
db.similarity_search(query)

# Week 3

We'll now try making an agent. We'll start by downloading a library to let us run a local language model.

In [ ]:
!pip install llama-cpp-python

Let's load the software.

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

With that loaded, let's download a language model.

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_0.gguf

Okay! Let's try loading it.

In [ ]:
llm = LlamaCpp(
    model_path="mistral-7b-instruct-v0.1.Q4_0.gguf",
    temperature=0.75,
    max_tokens=200,
    n_ctx=4096,
    top_p=1,
    f16_kv=True,
    verbose=True,
    callback_manager=callback_manager
)

Let's try it out!

In [ ]:
llm("Where is Tokyo?")

Having verified the language model works, let's try making a tool for it to access our vector store.

In [3]:
from langchain.chains import RetrievalQA

Let's define our agent.

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper

In [ ]:
tools = [
    Tool(
        name="Search",
        func=hamilton.run,
        description="useful for when you need to answer questions about the City of Hamilton. Returns documents concerning awardees and population. Input should be a type of award or particular district in Hamilton.",
    ),
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

And let's try it out.

In [ ]:
agent.run("Who, in Hamilton, won an award for art?")

# Week 4

Welcome back! We'll now try making an interface for our agent. First, let's download Gradio.

In [ ]:
!pip install gradio==3.48.0

Next, let's define a `message` function. This will let our Gradio interface talk to our agent.

In [ ]:
history = []

def message(mes, history):
    x = llm(mes)
    history.append(x)
    return x

With that done, let's try loading an interface in just a few lines of code.

In [ ]:
import gradio as gr

gr.ChatInterface(message, theme="soft", title="My Fancy Chatbot").launch()

Super! It works. We now can talk to an agent hooked up to some data through a fancy website.